In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-01 22:10:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.33MB/s    in 0.2s    

2022-05-01 22:10:01 (6.33 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [9]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

### Create DataFrames to match tables

In [16]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())
df.printSchema()

2402458
2402212
2402212
root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [24]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15247147|             1|
|   10069897|             1|
|   16127586|             3|
|    1688219|             1|
|    4578543|             1|
|   15960529|             1|
|   49084370|             2|
|   52951402|             2|
|   11337878|             8|
|   50714860|             6|
|   39134202|             2|
|    5501393|             1|
|   13542332|             4|
|   52136803|             1|
|   10450412|             1|
|   29151760|             2|
|   46333780|             1|
|   37666248|            10|
|   43060064|             2|
|   49483181|            36|
+-----------+--------------+
only showing top 20 rows



In [26]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0042M7PS2|Rodelle Gourmet B...|
|B0025UCHS6|Barilla Plus Past...|
|B00CJXSDZ0|Alert Fruit Energ...|
|B00CQBLPJ6|Alderwood Smoked ...|
|B000MN630Q|Taylors of Harrog...|
|B00BL0GJRG|Quest Bar Protein...|
|B001E8FD86|Atkins Dark Choco...|
|B00KOD31AE|Bradshaw #11991 2...|
|B00NO1SCS4|Send It Sweetly N...|
|B00IK0MSBQ|Organic Reishi Mu...|
|B003VIJI1A|Pacific Foods Org...|
|B00BAXAXRG|LorAnn Anise Oil,...|
|B002O3SAJE|Nescafe Espresso ...|
|B001EO5YQG|Classique Belgian...|
|B000MANIMK|Jolly Time Crispy...|
|B00EEJ54WC|Earth's Best Infa...|
|B005PI0RJG|Dizzy Pig BBQ Rag...|
|B00457LHCY|Sodastream Sodami...|
|B00JW3G3ZC|Arty Pure Natural...|
|B0000CDAY2|Downeast Lobster ...|
+----------+--------------------+
only showing top 20 rows



In [27]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100WFYHG5LBQZ|   46873148|B003PWC346|     822134010| 2014-11-06|
|R10174PSJAET60|   29532905|B00PT66JYQ|     749978411| 2015-01-09|
|R101DUK4KGSZKO|   47594902|B000REGVC0|     245638153| 2008-02-18|
|R102BWOXH8BAGB|   21812686|B00M2OGS08|     715217277| 2015-02-25|
|R102FRHV3HVRYH|   20091223|B004EKHMXU|     783192559| 2014-05-26|
|R102FVV96K2XPX|   26262840|B00ANXWKU2|     580871360| 2014-09-30|
|R102NGGB4MYAUU|   15820623|B004IXYLK0|     428171752| 2014-05-31|
|R103DLW6KGTI1T|   12722360|B00O1A0E6U|     449370336| 2015-02-18|
|R103KHSB6CGXI6|    8814515|B00FX1QCQC|     710636277| 2014-10-07|
|R103STKNZXONLT|   11956527|B00JD8V2HA|     612411339| 2015-08-07|
|R103ZT2DYXD1PT|    8179664|B001KWZKXE|     869231679| 2013-07-12|
|R1050BKZKF25VY|   21416238|B000MNJNRQ|     309219413| 2014-03

In [28]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100WFYHG5LBQZ|          5|            0|          0|   N|                Y|
|R10174PSJAET60|          4|           14|         16|   N|                Y|
|R101DUK4KGSZKO|          5|            1|          1|   N|                N|
|R102BWOXH8BAGB|          5|            0|          0|   N|                Y|
|R102FRHV3HVRYH|          1|            2|          2|   N|                Y|
|R102FVV96K2XPX|          5|            0|          1|   N|                Y|
|R102NGGB4MYAUU|          5|            1|          1|   N|                Y|
|R103DLW6KGTI1T|          5|            4|          4|   N|                Y|
|R103KHSB6CGXI6|          5|            0|          0|   N|                N|
|R103STKNZXONLT|          5|            2|          2|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [31]:
#Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://reviewdb.cfpf6rzmww9e.us-east-1.rds.amazonaws.com:5432/classdb"
config = {"user":"root", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [32]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [33]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [34]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [35]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)